In [11]:
from tqdm import tqdm

In [ ]:
grouped = df.groupby('data')

idx = 0
for book, group in tqdm(grouped):
    # Concatenate all sentences into a single string for each book
    full_text = ' '.join(group[' Sentence'])
    
    # Define the filename, using the book name. You might want to adjust this for valid filenames.
    filename = f"texts2/{idx}.txt"
    
    # Write the concatenated text to a file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(full_text)
        
    print(f"Written '{filename}' with {len(full_text.split())} words.")
    idx += 1

In [ ]:
import os
import pandas as pd
from nltk.tokenize import sent_tokenize

# Ensure NLTK is set up with the necessary components
import nltk
nltk.download('punkt')

def chunk_text_into_blocks(text, max_words=400, overlap=True):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_word_count = 0

    for i, sentence in enumerate(sentences):
        words = sentence.split()
        current_word_count += len(words)
        current_chunk.append(sentence)
        
        # If the current word count exceeds the limit or it's the last sentence
        if current_word_count >= max_words or i == len(sentences) - 1:
            # Add the current chunk to the list of chunks
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence] if overlap else []  # Start new chunk with overlap if enabled
            current_word_count = len(words)  # Reset word count

    return chunks

# Assuming your text files are named after the books and stored in a directory named 'books'
book_files = [f for f in os.listdir('texts2') if f.endswith('.txt')]
data = []

for book_file in tqdm(book_files):
    with open(f"texts2/{book_file}", 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Chunk the book's text
    chunks = chunk_text_into_blocks(text, 400)
    
    # Add chunks to the data list, along with the book name for identification
    book_name = book_file.replace('.txt', '').replace('_', ' ')
    for i, chunk in enumerate(chunks):
        data.append({'book': book_name, 'chunk_id': i + 1, 'text': chunk})

# Convert the list of dictionaries into a DataFrame
df_chunks = pd.DataFrame(data)

# Display the first few rows of the DataFrame to verify
print(df_chunks.head())
